In [34]:
# 決定木以外の方法でモデルを作ってみる

import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# データの読み込み
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# データの確認
display(train.dtypes)
display(test.dtypes)

# データ全体の確認
display(train.describe())
display(test.describe())

# 欠損データの確認
kesson_train = train.isnull().sum()
kesson_test = test.isnull().sum()
display(kesson_train)
display(kesson_test)



PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [35]:
# 文字列を数値に変換
train.loc[train["Sex"] == "male", "Sex"] = 0
train.loc[train["Sex"] == "female", "Sex"] = 1
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2

test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Sex"] == "female", "Sex"] = 1
test.loc[test["Embarked"] == "S", "Embarked"] = 0
test.loc[test["Embarked"] == "C", "Embarked"] = 1
test.loc[test["Embarked"] == "Q", "Embarked"] = 2

In [36]:
# 欠損データを埋める(train.csv)
# 年齢を目的関数として予測モデルを作る
target = 'Age'
FEATURE_COLS = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Sex']
features = FEATURE_COLS

# ageがあるものを訓練データに、ないものをテストデータにする
age_train_data = train[train[target].notnull()].copy()
age_test_data = train[train[target].isnull()].copy()

# 訓練データから検証データを分割して作成
random_seed = 228
X_train, X_val, y_train, y_val = train_test_split(
    age_train_data[features],
    age_train_data[target],
    test_size=0.2,
    random_state=random_seed
)

# テストデータの説明変数を設定
X_age_test = age_test_data[features]

# パラメータ設定
parameters = {
    "n_estimators" : [2, 5, 10, 15, 20, 30, 50, 75, 100],
    "criterion" : ["squared_error"],
    "min_samples_split" : [2, 3, 5, 10],
    "max_depth" : [2, 3, 5, 10],
    "max_features" : [1, 2, 3, 4, 5],
    "random_state" : [random_seed],
    "verbose" : [False],
}

# ランダムフォレストで予測モデルを作成
age_model = GridSearchCV(RandomForestRegressor(), parameters, cv=3)
age_model.fit(X_train, y_train)
age_model = age_model.best_estimator_

# 検証データで精度を確認
age_model.score(X_val, y_val)
val_score = age_model.score(X_val, y_val)
print(f"Age予測モデルの検証データに対する精度: {val_score}")

predicted_train_ages = age_model.predict(X_age_test)

# 予測した年齢で欠損値を埋める
train.loc[(train['Age'].isnull()), 'Age'] = predicted_train_ages

Age予測モデルの検証データに対する精度: 0.2576451600482287


In [15]:
# 欠損データを埋める(test.csv)
# Fareの欠損値を中央値で埋める
test['Fare'].fillna(test['Fare'].median())

# 作成した予測モデルでAgeの欠損値を埋める
test_age_test_data = test[test['Age'].isnull()].copy()
X_test_age_test = test_age_test_data[features]

predicted_test_ages = age_model.predict(X_test_age_test)

test.loc[(test['Age'].isnull()), 'Age'] = predicted_test_ages

In [17]:
# Survivedを目的変数、その他を説明変数として設定
target = 'Survived'
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

X_train = train[features]
y_train = train[target]
X_test = test[features]

# ランダムフォレストで予測モデルを作成
model = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=228)
model.fit(X_train, y_train)
predicted = model.predict(X_test)

# 結果の保存
PassengerId = np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(data=predicted, index=PassengerId, columns = ["Survived"])
my_solution.to_csv("titanic_RandomForest_solution.csv", index_label = ["PassengerId"])